# Image Embedding

![](./img/model_architecture.png)

<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    "ViT architecture"
  </figcaption>

가장 먼저 만나게 되는 Image Embedding 파트 입니다!

다음 4단계로 Image 를 Embedding 합니다.
- Image 를 Patch 라는 단위로 나누고 Flatten 하기
- Flatten 된 Vector 에 Class token 을 추가하기
- Class token 이 추가된 Vector 에 Positional Embedding 더하기 

> 추가적으로, Einops package 에 대한 내용을 같이 작성하려고 하다가, 너무 길어져서 따로 빼 두었습니다.\
> 이 내용은 `A1-Einops_toturial_for_vit.ipynb` 에서 확인할 수 있습니다.\
> 부가적인 부분이지만 내용의 흐름상 꼭 참고해주시면 좋겠습니다:)

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

In [2]:
ims = torch.Tensor(np.load('./resources/test_images.npy', allow_pickle=False))
ims = rearrange(ims, 'b h w c -> b c h w')
print(type(ims), ims.shape)

<class 'torch.Tensor'> torch.Size([6, 3, 96, 96])


여기서 차원은, `matplotlib` 에 친화적인 `b h w c` 가 아닌

`pytorch` 친화적인 `b c h w` 로 사용합니다.

In [3]:
patch_size = 16 # 16 pixels

print('ims :', ims.shape)
patches = rearrange(ims, 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size, s2=patch_size)
print('patches :', patches.shape)

ims : torch.Size([6, 3, 96, 96])
patches : torch.Size([6, 36, 768])


----
## Image 를 Patch 라는 단위로 나누고 Flatten 하기


einops의 rearrange를 이용하여
$$
Batch \times Channel \times Hight \times Width 
$$
를 가진 텐서를 
$$
Batch \times \text{num of patch} \times \text{patch size} (\cal{flattened})
$$
로 변경해 줍니다. 

이 과정을 정리해 보면, 총 $Batch$ 개의 이미지를 아래와 같이 patch size 로 자른 뒤, 

<center>

![](./img/split_e.png)

</center>
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    fig 1: image to patch
  </figcaption>

flatten 을 시키는 것이라고 생각하시면 됩니다. 


### in-and-out
>```python
>ims : torch.Size([6, 3, 96, 96])
>patches : torch.Size([6, 36, 768])
>```


결론적으로, 각 이미지당 patch 의 개수는 36개 이고, 각 patch 를 구성하는 pixel 은 768개 임을 알 수 있습니다.

이를 모델에 적용하게 되면 다음과 같이 사용할 수 있습니다.

----
### 1. Einops 를 사용하는 방법

In [4]:
patch_size = 16
in_channels = 3
hidden_dim = patch_size*patch_size*in_channels
embedding_dim = 768

process_input = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, embedding_dim),
            nn.LayerNorm(embedding_dim),
        )

image_embedded1 = process_input(ims)
print(image_embedded1.shape)

torch.Size([6, 36, 768])


여기서 `embedding_dim` 은, hyper-parameter 로 사용하여 자유도를 설정할 수 있습니다만, 

우선은 이해를 돕기 위해 `hidden_dim` (`patch_size` 에 의해 픽셀 위치만 바뀌어 설정되는 크기) 과 동일하게 설정하겠습니다. 

----
### 2. Conv2d를 사용하는 공식 문서

실제 pytorch 공식 문서에는, 다음과 같이 구성이 되어 있습니다. 

```python
def _process_input(self, x: torch.Tensor) -> torch.Tensor:
        n, c, h, w = x.shape
        p = self.patch_size
        torch._assert(h == self.image_size, f"Wrong image height! Expected {self.image_size} but got {h}!")
        torch._assert(w == self.image_size, f"Wrong image width! Expected {self.image_size} but got {w}!")
        n_h = h // p
        n_w = w // p

        # (n, c, h, w) -> (n, hidden_dim, n_h, n_w)
        x = self.conv_proj(x)
        # (n, hidden_dim, n_h, n_w) -> (n, hidden_dim, (n_h * n_w))
        x = x.reshape(n, self.hidden_dim, n_h * n_w)

        # (n, hidden_dim, (n_h * n_w)) -> (n, (n_h * n_w), hidden_dim)
        # The self attention layer expects inputs in the format (N, S, E)
        # where S is the source sequence length, N is the batch size, E is the
        # embedding dimension
        x = x.permute(0, 2, 1)

        return x
```

만, 이를 직관적으로 다시 구성해보죠.

In [5]:
def process_input(x: torch.Tensor, hidden_dim: int = None) -> torch.Tensor:
        n, c, h, w = x.shape
        p = patch_size
        n_h = h // p
        n_w = w // p
        
        if hidden_dim == None:
                hidden_dim = p * p * c

        conv_proj = nn.Sequential(
                nn.LayerNorm(
                        [c, h, w]
                ),
                nn.Conv2d(
                in_channels=3, out_channels=hidden_dim, kernel_size=patch_size, stride=patch_size
                ),
                nn.LayerNorm(
                        [hidden_dim, n_h, n_w]
                )
        )
        
        # (n, c, h, w) -> (n, hidden_dim, n_h, n_w)
        x = conv_proj(x)
        # (n, hidden_dim, n_h, n_w) -> (n, hidden_dim, (n_h * n_w))
        x = x.reshape(n, hidden_dim, n_h * n_w)

        # (n, hidden_dim, (n_h * n_w)) -> (n, (n_h * n_w), hidden_dim)
        # The self attention layer expects inputs in the format (N, S, E)
        # where S is the source sequence length, N is the batch size, E is the
        # embedding dimension
        x = x.permute(0, 2, 1)

        return x

image_embedded2 = process_input(ims)

print('image_embedded1 shape: ', image_embedded1.shape)
print('image_embedded2 shape: ', image_embedded2.shape)

image_embedded1 shape:  torch.Size([6, 36, 768])
image_embedded2 shape:  torch.Size([6, 36, 768])


여기서도, 이해를 돕기 위해 hyper-parpameter 인 `hidden_dim` 을 pixel 개수에 변화가 없도록 지정해 두었습니다. 

결론적으로, 위와 같이 동일한 출력값을 가지고 있다는 것을 확인할 수 있습니다. 

두 코드의 차이는 linear layer 로 구성했는지 와, Conv2d layer 로 구성했는지의 차이 입니다. 

개인적으로 Einops 를 사용한 코드가 직관적이라고 생각하여 작성해 보았습니다. 

----
## Flatten 된 Vector 에 Class token 을 추가하기

출력으로 가지고 있는 `torch.Size([6, 36, 768])` vector 의 의미를 다시 한 번 생각해 보죠.
>```python
>torch.Size([              6,              36,                         768])
>torch.Size([number of image, number of patch, hidden_dim(hyper-parameter)])
>```


여기서, 각 이미지에 대한 class 를, 학습할 수 있는 추가적인 parameter 를 사용하려고 합니다. 

ViT 논문에서는, 각 image 당 한장의 patch 를 추가하여 class 를 대변하려고 하고, 이를 `Class token` 이라고 설명합니다. 

In [6]:
n = ims.shape[0] # number of image
print('image_embedded2 shape: ', image_embedded2.shape)
class_token = nn.Parameter(torch.zeros(1, 1, hidden_dim))
batch_class_token = class_token.expand(n, -1, -1)
image_class_embedded = torch.cat([batch_class_token, image_embedded2], dim=1)
print('image_class_embedded shape: ', image_class_embedded.shape)


image_embedded2 shape:  torch.Size([6, 36, 768])
image_class_embedded shape:  torch.Size([6, 37, 768])


위와 같은 방식으로 말이죠.

patch 차원에 1개의 patch 가 추가된 것을 알 수 있습니다. 

이렇게 image 자체를 특정 embedding space 에 vector 형태로 embedding 했다면, 

이후는 NLP 와 비슷한 방식으로 흘러 간다고 보시면 됩니다. 

그 시작으로, Positional embedding 을 적용해 봅시다.

---
## Class token 이 추가된 Vector 에 Positional Embedding 더하기 

pytorch 공식 문서에는 다음과 같이 pos_embedding을 지정하여 사용 합니다. 

In [7]:
seq_length = 36 # Number of patchs
pos_embedding = nn.Parameter(torch.empty(1, seq_length+1, hidden_dim).normal_(std=0.02))  # from BERT

final_embedded = image_class_embedded + pos_embedding

print('image_class_embedded shape: ', image_class_embedded.shape)
print('final_embedded shape: ', final_embedded.shape)

image_class_embedded shape:  torch.Size([6, 37, 768])
final_embedded shape:  torch.Size([6, 37, 768])


-----
# 정리

전체 모델을 돌아보면, 지금까지 구현된 부분이 어느정도 일까요?

<center>

![](./img/image_embedding.png)

</center>
<figcaption style="text-align:center; font-size:15px; color:#808080; margin-top:40px">
    fig 2: image embedding
  </figcaption>

모든 과정을 돌아보면 전체 그림에서 fig 2에 해당하는 부분을 마무리 했다는 것을 알 수 있을 겁니다.

추가적으로, 처음 봤을 때 의문이었던, Class token 과 patch 로 나눈 이미지를 sequance 로 사용하는 것이 이해가 되면 좋겠네요.

다음은, 본격적으로 Transformer 구조를 구현해 봅시다!



> 우선 큰 방향성은, 
> 
> 각 모듈에 대한 뼈대를 구축하고 
> 
> 각 모듈에 대한 구현으로 연결해 나가 보도록 하죠.